#Importando bases

In [ ]:
# KAGGLE: https://www.kaggle.com/code/andrscampaa/detecting-bullying-tweets-pytorch-lstm-bert/edit
# Tutorial: https://www.youtube.com/watch?v=wp9BudYGZyA&t=1862s
# Actualizando matplotlib
#!pip install matplotlib==3.4
#import os
#os.kill(os.getpid(), 9)

In [ ]:
# Descargando librerías necesarias
!pip install transformers
!pip install -U deep-translator
!python -m spacy download es_core_news_md
!python -m spacy download en_core_web_md

import matplotlib
import re
import es_core_news_md
import en_core_web_md
import pandas as pd 
import base64
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
def create_onedrive_directdownload (onedrive_link):
    data_bytes64 = base64.b64encode(bytes(onedrive_link, 'utf-8'))
    data_bytes64_String = data_bytes64.decode('utf-8').replace('/','_').replace('+','-').rstrip("=")
    resultUrl = f"https://api.onedrive.com/v1.0/shares/u!{data_bytes64_String}/root/content"
    return resultUrl

# Input any OneDrive URL 
onedrive_url = "https://1drv.ms/x/s!AhJYFCPR8DpKiLluuU5Ia5sGNc7Jew?e=hoETSw"
# Generate Direct Download URL from above Script
direct_download_url = create_onedrive_directdownload(onedrive_url)
print(direct_download_url)

pp=0
try:
  dff = pd.read_excel(direct_download_url)
  print("Finale...")
  dff=dff.loc[~pd.isnull(dff["tweet_text_translated_clean"])]
  dff=dff.loc[~pd.isnull(dff["tweet_text_clean"])]
  dff.loc[dff["cyberbullying_type"]=="not_cyberbullying","cyberbullying_type2"]=0
  dff.loc[dff["cyberbullying_type"]!="not_cyberbullying","cyberbullying_type2"]=1
  dff.loc[dff["cyberbullying_type"]=="not_cyberbullying","cyberbullying_typex"]="not_cyberbullying"
  dff.loc[dff["cyberbullying_type"]!="not_cyberbullying","cyberbullying_typex"]="cyberbullying"
  dff.loc[dff["cyberbullying_type"]=="not_cyberbullying","cyberbullying_type3"]=0
  dff.loc[dff["cyberbullying_type"]=="age","cyberbullying_type3"]=1
  dff.loc[dff["cyberbullying_type"]=="gender","cyberbullying_type3"]=2
  dff.loc[dff["cyberbullying_type"]=="ethnicity","cyberbullying_type3"]=3
  dff.loc[dff["cyberbullying_type"]=="religion","cyberbullying_type3"]=4
  dff.loc[dff["cyberbullying_type"]=="other_cyberbullying","cyberbullying_type3"]=5
  dff = dff[~dff["tweet_text_clean"].duplicated()]
  dff = dff[~dff["tweet_text_translated_clean"].duplicated()]
  pp = 1
except:
  pass

if pp!=1:
  # Input any OneDrive URL (original)
  onedrive_url = "https://1drv.ms/u/s!AhJYFCPR8DpKiK4c1lzD4HoS8zDIQQ?e=poWnUK"
  # Generate Direct Download URL from above Script
  direct_download_url = create_onedrive_directdownload(onedrive_url)
  print(direct_download_url)

  # Load Dataset to the Dataframe (traducida)
  df = pd.read_csv(direct_download_url)
  df["prediccion_jonatan"]=""
  print(df.columns)
  print(len(df))

  # Input any OneDrive URL 
  onedrive_url = "https://1drv.ms/x/s!AhJYFCPR8DpKiLZZbc3wCPMCbbHQzw?e=nF6YEf"
  # Generate Direct Download URL from above Script
  direct_download_url = create_onedrive_directdownload(onedrive_url)
  print(direct_download_url)

  try:
    df2 = pd.read_excel(direct_download_url)
    dff = pd.concat([df,df2["tweet_text_translated"]],axis=1)
    dff["Unnamed: 0"]=dff.index
  except:
    pass

print("Matplotlib version:",matplotlib.__version__)
print(pp)
try:
  print("Missings en tweets traducidos:",dff["tweet_text_translated_clean"].isna().sum())
  print("Missings en tweets:",dff["tweet_text_clean"].isna().sum())
  print("Duplicados de tweets:",dff["tweet_text_clean"].duplicated().sum())
  print("Duplicados de tweets: traducidos:",dff["tweet_text_translated_clean"].duplicated().sum())
  print("Base final tiene:",len(dff))
except:
  pass

#Seleccionando base

In [ ]:
from sklearn.model_selection import train_test_split
from deep_translator import GoogleTranslator
#from google.colab import files

if pp!=1:
  if "tweet_text_translated" not in dff.columns.tolist():
    print("no hay")
    df["tweet_text_translated"]=""
    X = df[['tweet_text','tweet_text_translated','prediccion_jonatan']]
    y=df['cyberbullying_type']
    z=""

    # Traducción
    if z=="":
      for j in [47000,48000]:
        p=1
        for i in range(j-1000,j):#len(X)):
          try:
            print(p)
            translated = GoogleTranslator(source='auto', target='es').translate(X["tweet_text"][i])
            X["tweet_text_translated"][i]=translated
            p+=1
            if i+1==j:
              jj=j-1000
              data = X.iloc[jj:j,:]
              data.to_excel("aqui"+str(i)+".xlsx")
              #files.download("aqui"+str(i)+".xlsx")
          except:
            p+=1
            pass

    else:
      pass

  else:
    dff=dff.loc[~pd.isnull(dff["tweet_text_translated"])]
    print("Missings traducidos:",dff.isna().sum())
    dff["prediccion_jonatan"] = ""
    dff["Unnamed: 0"] = dff.index
    X = dff[["Unnamed: 0",'tweet_text','tweet_text_translated','prediccion_jonatan']]
    dff["cyberbullying_type2"] = dff["cyberbullying_type"]
    dff.loc[dff["cyberbullying_type"]!="not_cyberbullying","cyberbullying_type"]="Bullying"
    dff.loc[dff["cyberbullying_type"]=="not_cyberbullying","cyberbullying_type"]="Not_bullying"
    dff.loc[dff["cyberbullying_type"]=="Bullying","cyberbullying_type"]=1
    dff.loc[dff["cyberbullying_type"]=="Not_bullying","cyberbullying_type"]=0
    y = dff["cyberbullying_type"].astype(int)

# Limpiando nulls
print(len(dff))
print("casi listo")
X = dff[["Unnamed: 0",'tweet_text','tweet_text_translated','prediccion_jonatan','tweet_text_translated_clean','tweet_text_clean']]
yyy = dff[["cyberbullying_type2","cyberbullying_type3"]].astype(int)

# Luego de ver resultados, igual se debe verificar con train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, yyy, test_size=0.2,random_state=17)
print("Hay por procesar:", len(X_test))
print(X_test.head())

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

## Funciones

In [ ]:
# Pre procesamiento para NUEVA DATA
def get_features(rev):
    obj = []
    doc = re.sub(r'\W', ' ', rev)
    doc = re.sub(r'\s+[a-zA-Z]\s+', ' ', doc) 
    doc = re.sub(r'\^[a-zA-Z]\s+', ' ', doc) 
    doc = re.sub(r'\s+', ' ', doc, flags=re.I)
    doc = re.sub(r'^b\s+', '', doc)
    doc = doc.lower()
    doc = re.sub('rt', '', doc)
    doc = re.sub('él', '', doc)
    doc = re.sub('co', '', doc)
    doc = re.sub('_', '', doc) 
    doc = nlp (doc)
    lemmas = [no_stop.lemma_ for no_stop in doc if not no_stop.is_stop]
    doc = ' '.join(lemmas)
    doc = re.sub('él', '', doc)
    doc = re.sub(r'\s+[a-zA-Z]\s+', ' ', doc)
    doc = re.sub(r'\^[a-zA-Z]\s+', ' ', doc)
    obj.append(doc)
    obj = vectorizer.transform(obj)
    obj = tfidfconverter.transform(obj)
    return obj

def cooler_confusion_matrix(y, y_pred, title, labels):
    fig, ax =plt.subplots(figsize=(7.5,7.5))
    ax=sns.heatmap(confusion_matrix(y, y_pred), annot=True, cmap="Blues", fmt='g', cbar=False, annot_kws={"size":30})
    plt.title(title, fontsize=25)
    ax.xaxis.set_ticklabels(labels, fontsize=16) 
    ax.yaxis.set_ticklabels(labels, fontsize=14.5)
    ax.set_ylabel('Test', fontsize=25)
    ax.set_xlabel('Predicción', fontsize=25)
    plt.show()

# Bi clase

#Español

### Estadística descriptiva

In [ ]:
# Modelando
f1=[]
acc=[]
prec=[]
spec=[]
sens=[]
vpp=[]
vpn=[]
auc=[]
mod=[]
dat=[]

def autopct_format(values):
        def my_format(pct):
            total = sum(values)
            val = int(round(pct*total/100.0))
            return '{:.1f}%\n({v:d})'.format(pct, v=val)
        return my_format

# Pie 1
labels = ['religión', 'edad', 'género', 'etnicidad', 'no cyberbullying', 'otro tipo de cyberbullying']
s = dff.cyberbullying_type.value_counts()
plt.pie(s,labels = labels, autopct=autopct_format(s))
plt.show()

# Pie 2
labels = ['cyberbullying', 'no cyberbullying']
s = dff.cyberbullying_typex.value_counts()
plt.pie(s,labels = labels, autopct=autopct_format(s))
plt.show()

# Longitud
text_len = []
for text in dff["tweet_text_translated"]:
    tweet_len = len(text.split())
    text_len.append(tweet_len)

dff['text_translated_orig_len'] = text_len
plt.figure(figsize=(7,5))
ax = sns.countplot(x='text_translated_orig_len', data=dff, palette='magma')
plt.title('Conteo de textos originales por número de palabras (español)', fontsize=20)
plt.yticks([])
plt.xticks([])
#for i in ax.containers:
#    ax.bar_label(i,)
plt.ylabel('conteo')
plt.xlabel('')
plt.show()
print(dff['text_translated_orig_len'].describe())

from collections import Counter
results = set()
dff['tweet_text_translated'].str.lower().str.split().apply(results.update)
results = Counter()
dff['tweet_text_translated'].str.lower().str.split().apply(results.update)
print(len(results))
print(results.most_common())

### Hugging face

In [ ]:
from transformers import pipeline
model_path = "JonatanGk/roberta-base-bne-finetuned-cyberbullying-spanish"
bullying_analysis = pipeline("text-classification", model=model_path, tokenizer=model_path)

p=1
for i in X_test.index:
  try:
    print(p)
    X_test["prediccion_jonatan"][i]=bullying_analysis(X_test["tweet_text_translated"][i])[0]["label"]
    p+=1
  except:
    p+=1
    pass

from google.colab import files
data=X_test.join(y_test)
data.loc[data["prediccion_jonatan"]=="Bullying","prediccion_jonatan"]=1
data.loc[data["prediccion_jonatan"]=="Not_bullying","prediccion_jonatan"]=0
data.loc[data["prediccion_jonatan"]=="","prediccion_jonatan"]=0
data["prediccion_jonatan"]=data["prediccion_jonatan"].fillna(0)

print("Resultados Hugging face...")
print(cooler_confusion_matrix(data["cyberbullying_type2"].astype(int),data["prediccion_jonatan"].astype(int),"Aqui",["Si","No"]))
print(classification_report(data["cyberbullying_type2"].astype(int),data["prediccion_jonatan"].astype(int)))

mod.append("Hugging Face")
f1.append(round(f1_score(data["cyberbullying_type2"].astype(int),data["prediccion_jonatan"].astype(int), average='weighted'),2))
acc.append(round(accuracy_score(data["cyberbullying_type2"].astype(int),data["prediccion_jonatan"].astype(int)),2))
auc.append(round(roc_auc_score(data["cyberbullying_type2"].astype(int),data["prediccion_jonatan"].astype(int)),2))
prec.append(round(precision_score(data["cyberbullying_type2"].astype(int),data["prediccion_jonatan"].astype(int)),2))
tn, fp, fn, tp = confusion_matrix(data["cyberbullying_type2"].astype(int),data["prediccion_jonatan"].astype(int)).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
spec.append(round(specificity,2))
sens.append(round(sensitivity,2))
vpps = tp / (tp+fp)
vpns = tn / (tn+fn)
vpp.append(round(vpps,2))
vpn.append(round(vpns,2))

p=1
for i in X_test.index:
  try:
    print(p)
    X_test["prediccion_jonatan"][i]=bullying_analysis(X_test["tweet_text_translated_clean"][i])[0]["label"]
    p+=1
  except:
    p+=1
    pass

from google.colab import files
data=X_test.join(y_test)
data.loc[data["prediccion_jonatan"]=="Bullying","prediccion_jonatan"]=1
data.loc[data["prediccion_jonatan"]=="Not_bullying","prediccion_jonatan"]=0
data.loc[data["prediccion_jonatan"]=="","prediccion_jonatan"]=0
data["prediccion_jonatan"]=data["prediccion_jonatan"].fillna(0)

print("Resultados Hugging face...")
print(cooler_confusion_matrix(data["cyberbullying_type2"].astype(int),data["prediccion_jonatan"].astype(int),"Aqui",["Si","No"]))
print(classification_report(data["cyberbullying_type2"].astype(int),data["prediccion_jonatan"].astype(int)))

mod.append("Hugging Face clean")
f1.append(round(f1_score(data["cyberbullying_type2"].astype(int),data["prediccion_jonatan"].astype(int), average='weighted'),2))
acc.append(round(accuracy_score(data["cyberbullying_type2"].astype(int),data["prediccion_jonatan"].astype(int)),2))
auc.append(round(roc_auc_score(data["cyberbullying_type2"].astype(int),data["prediccion_jonatan"].astype(int)),2))
prec.append(round(precision_score(data["cyberbullying_type2"].astype(int),data["prediccion_jonatan"].astype(int)),2))
tn, fp, fn, tp = confusion_matrix(data["cyberbullying_type2"].astype(int),data["prediccion_jonatan"].astype(int)).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
spec.append(round(specificity,2))
sens.append(round(sensitivity,2))
vpps = tp / (tp+fp)
vpns = tn / (tn+fn)
vpp.append(round(vpps,2))
vpn.append(round(vpns,2))

### TF-IDF

In [ ]:
nlp = es_core_news_md.load()
documents = []
j = 0
X = dff[["Unnamed: 0",'tweet_text','tweet_text_translated','prediccion_jonatan']]
XX = X["tweet_text_translated"]
y = yyy["cyberbullying_type2"]

if pp!=1:
  for sen in XX.index:
      # Removiendo caracteres especiales
      document = re.sub(r'\W', ' ', str(XX[sen]))
      
      # Removiendo caracteres solitarios despues del inicio
      document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
      
      # Removiendo caracteres solitarios al inicio
      document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

      # Removiendo el prefijo b
      document = re.sub(r'^b\s+', '', document) #solo cuando se agrega al inicio
      
      # Minúsculas
      document = document.lower()

      # Removiendo caracteres solitarios al inicio
      document = re.sub('rt', '', document)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('http', '', document)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('co', '', document)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('_', '', document) 

      # Removiendo multiples espacios
      document = re.sub(r'\s+', ' ', document, flags=re.I)
      
      # Lematizacion
      document = nlp (document)
      lemmas = [no_stop.lemma_ for no_stop in document if not no_stop.is_stop]
      document = ' '.join(lemmas)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('él', '', document)

      # Removiendo caracteres solitarios despues del inicio
      document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
      
      # Removiendo caracteres solitarios al inicio
      document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)
  
      documents.append(document)
      j = j+1
      print(j)

  # Longitud
  text_len = []
  for text in documents:
      tweet_len = len(text.split())
      text_len.append(tweet_len)

  dff["tweet_text_translated_clean"]=documents
  dff['text_translated_clean_len'] = text_len
  dff=dff.loc[~pd.isnull(dff["tweet_text_translated_clean"])]

plt.figure(figsize=(7,5))
ax = sns.countplot(x='text_translated_clean_len', data=dff, palette='magma')
plt.title('Conteo de textos limpiados por número de palabras (español)', fontsize=20)
plt.yticks([])
plt.xticks([])
#for i in ax.containers:
#    ax.bar_label(i,)
plt.ylabel('conteo')
plt.xlabel('')
plt.show()
print(dff['text_translated_clean_len'].describe())

from collections import Counter
results = set()
dff['tweet_text_translated_clean'].str.lower().str.split().apply(results.update)
results = Counter()
dff['tweet_text_translated_clean'].str.lower().str.split().apply(results.update)
print(len(results))
print(results.most_common())

yy = [count for tag, count in results.most_common(20)]
x = [tag for tag, count in results.most_common(20)]

plt.barh(x,yy, color='crimson')

# Vectorizar
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1000, min_df=5, max_df=0.7)
XX = vectorizer.fit_transform(dff['tweet_text_translated_clean']).toarray()

# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
XX = tfidfconverter.fit_transform(XX).toarray()

# Separando (equivalente al de arriba por el random state)
X_train, X_test, y_train, y_test = train_test_split(XX, y, test_size=0.2,random_state=17)
print("Hay por procesar:", len(X_test))

###  + RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Perra loca"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de bullying:", rf.predict_proba(review1_features)[0,1]) #0 es por la primera obs, y 1 es por prob de posi

mod.append("TF-IDF + RF")
f1.append(round(f1_score(y_test,y_pred, average='weighted'),2))
acc.append(round(accuracy_score(y_test,y_pred),2))
auc.append(round(roc_auc_score(y_test,y_pred),2))
prec.append(round(precision_score(y_test,y_pred),2))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
spec.append(round(specificity,2))
sens.append(round(sensitivity,2))
vpps = tp / (tp+fp)
vpns = tn / (tn+fn)
vpp.append(round(vpps,2))
vpn.append(round(vpns,2))

### + SVM

In [ ]:
from sklearn.svm import SVC
sv = SVC(random_state=17)
sv.fit(X_train, y_train)
y_pred = sv.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Perra loca"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de bullying:", sv.predict(review1_features.toarray())) #0 es por la primera obs, y 1 es por prob de posi

mod.append("TF-IDF + SVM")
f1.append(round(f1_score(y_test,y_pred, average='weighted'),2))
acc.append(round(accuracy_score(y_test,y_pred),2))
auc.append(round(roc_auc_score(y_test,y_pred),2))
prec.append(round(precision_score(y_test,y_pred),2))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
spec.append(round(specificity,2))
sens.append(round(sensitivity,2))
vpps = tp / (tp+fp)
vpns = tn / (tn+fn)
vpp.append(round(vpps,2))
vpn.append(round(vpns,2))

### + Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Puta"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de bullying:", nb.predict_proba(review1_features.toarray())[0,1]) #0 es por la primera obs, y 1 es por prob de posi

mod.append("TF-IDF + Naive Bayes")
f1.append(round(f1_score(y_test,y_pred, average='weighted'),2))
acc.append(round(accuracy_score(y_test,y_pred),2))
auc.append(round(roc_auc_score(y_test,y_pred),2))
prec.append(round(precision_score(y_test,y_pred),2))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
spec.append(round(specificity,2))
sens.append(round(sensitivity,2))
vpps = tp / (tp+fp)
vpns = tn / (tn+fn)
vpp.append(round(vpps,2))
vpn.append(round(vpns,2))

### + Redes neuronales

In [ ]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes=(150,100,50,10), max_iter=300, activation = 'relu',solver='sgd',random_state=17)
nn.fit(X_train, y_train)
y_pred = nn.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Puta"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de bullying:", nn.predict_proba(review1_features.toarray())[0,1]) #0 es por la primera obs, y 1 es por prob de posi

mod.append("TF-IDF + Redes Neuronales")
f1.append(round(f1_score(y_test,y_pred, average='weighted'),2))
acc.append(round(accuracy_score(y_test,y_pred),2))
auc.append(round(roc_auc_score(y_test,y_pred),2))
prec.append(round(precision_score(y_test,y_pred),2))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
spec.append(round(specificity,2))
sens.append(round(sensitivity,2))
vpps = tp / (tp+fp)
vpns = tn / (tn+fn)
vpp.append(round(vpps,2))
vpn.append(round(vpns,2))

### BETO

In [ ]:
### Preparación general
# Librerías para BETO
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
modelo='dccuchile/bert-base-spanish-wwm-uncased'
variable_x="tweet_text_translated"
nombre_modelo='bullying_beto_modelo'
z=16 # Batch size
epocas=3 # N° épocas

X = dff[["Unnamed: 0",'tweet_text','tweet_text_translated','prediccion_jonatan']]
# train-test-split para documents (textos sin íconos especiales ni doble espaciado)
X_train, X_test, y_train, y_test = train_test_split(X[variable_x], y, test_size=0.2,random_state=17)
print("Hay por procesar:", len(X_test))
print(X_test)

# Trayendo al modelo de Hugging Face
tokenizer = BertTokenizer.from_pretrained(modelo)

# Tokenizando
token = tokenizer.encode_plus(
    X_train.iloc[0],
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)
print(token.input_ids)
X_input_ids = np.zeros((len(X_train), 256))
X_attn_masks = np.zeros((len(X_train), 256))

# Funciones
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(X_train)):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

# Creando efectivamente la tokenización
X_input_ids, X_attn_masks = generate_training_data(X_train, X_input_ids, X_attn_masks, tokenizer)
labels = np.zeros((len(X_train), 2))
labels[np.arange(len(X_train)), y_train] = 1

# Pasando a tensorflow form
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
#dataset.take(2) # one sample data

def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

### Preparación modelo
# Seteando modelo
print("Batch size de:",z,"\nNúmero de épocas de:",epocas)
dataset = dataset.map(SentimentDatasetMapFunction)
print(len(dataset))
dataset = dataset.batch(z, drop_remainder=True)
print(len(dataset))
p = 0.8 # Porcentaje de datos para entrenar
train_size = int((len(X_train)//z)*p)
train_dataset = dataset.take(train_size)
print(len(train_dataset))
val_dataset = dataset.skip(train_size)
print(len(val_dataset))

from transformers import TFBertModel
model = TFBertModel.from_pretrained(modelo)

# Hay 2 inputs, los id y los attention mask
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

# Los embeddings
bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(2, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

# Arquitectura
sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

# Estimando modelo
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
accc = tf.keras.metrics.CategoricalAccuracy('accuracy')
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[accc])
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epocas
)

# Guardando
sentiment_model.save(nombre_modelo)

# Cargando
sentiment_model = tf.keras.models.load_model(nombre_modelo)
tokenizer = BertTokenizer.from_pretrained(modelo)

### Evaluando lo obtenido
# Funciones
def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['Not_bullying', 'Bullying']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

# Evaluando general
pd.DataFrame(hist.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

# Evaluando específico
input_text = 'Parecía carbón'
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")
!zip -r /content/file.zip /content/bullying_beto_modelo

y_pred = []
p=1
for i in X_test:
  #print(p)
  processed_data = prepare_data(i, tokenizer)
  result = make_prediction(sentiment_model, processed_data=processed_data)
  #print(f"Predicted Sentiment: {result}")
  if result=="Bullying":
    y_pred.append(1)
  else:
    y_pred.append(0)
  p+=1

# Para Excel
mod.append("BETO")
f1.append(round(f1_score(y_test,y_pred, average='weighted'),2))
acc.append(round(accuracy_score(y_test,y_pred),2))
auc.append(round(roc_auc_score(y_test,y_pred),2))
prec.append(round(precision_score(y_test,y_pred),2))
print(confusion_matrix(y_test, y_pred))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
spec.append(round(specificity,2))
sens.append(round(sensitivity,2))
vpps = tp / (tp+fp)
vpns = tn / (tn+fn)
vpp.append(round(vpps,2))
vpn.append(round(vpns,2))

In [ ]:
# Tutorial: https://www.youtube.com/watch?v=wp9BudYGZyA&t=1862s
### Preparación general
# Librerías para BETO
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
modelo='dccuchile/bert-base-spanish-wwm-uncased'
variable_x="tweet_text_translated_clean"
nombre_modelo='bullying_beto_modelo'
z=16 # Batch size
epocas=3 # N° épocas

X = dff[["Unnamed: 0",'tweet_text','tweet_text_translated_clean','prediccion_jonatan']]
# train-test-split para documents (textos sin íconos especiales ni doble espaciado)
X_train, X_test, y_train, y_test = train_test_split(X[variable_x], y, test_size=0.2,random_state=17)
print("Hay por procesar:", len(X_test))
print(X_test)

# Trayendo al modelo de Hugging Face
tokenizer = BertTokenizer.from_pretrained(modelo)

# Tokenizando
token = tokenizer.encode_plus(
    X_train.iloc[0],
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)
print(token.input_ids)
X_input_ids = np.zeros((len(X_train), 256))
X_attn_masks = np.zeros((len(X_train), 256))

# Funciones
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(X_train)):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

# Creando efectivamente la tokenización
X_input_ids, X_attn_masks = generate_training_data(X_train, X_input_ids, X_attn_masks, tokenizer)
labels = np.zeros((len(X_train), 2))
labels[np.arange(len(X_train)), y_train] = 1

# Pasando a tensorflow form
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
#dataset.take(2) # one sample data

def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

### Preparación modelo
# Seteando modelo
print("Batch size de:",z,"\nNúmero de épocas de:",epocas)
dataset = dataset.map(SentimentDatasetMapFunction)
print(len(dataset))
dataset = dataset.batch(z, drop_remainder=True)
print(len(dataset))
p = 0.8 # Porcentaje de datos para entrenar
train_size = int((len(X_train)//z)*p)
train_dataset = dataset.take(train_size)
print(len(train_dataset))
val_dataset = dataset.skip(train_size)
print(len(val_dataset))

from transformers import TFBertModel
model = TFBertModel.from_pretrained(modelo)

# Hay 2 inputs, los id y los attention mask
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

# Los embeddings
bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(2, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

# Arquitectura
sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

# Estimando modelo
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
accc = tf.keras.metrics.CategoricalAccuracy('accuracy')
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[accc])
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epocas
)

# Guardando
sentiment_model.save(nombre_modelo)

# Cargando
sentiment_model = tf.keras.models.load_model(nombre_modelo)
tokenizer = BertTokenizer.from_pretrained(modelo)

### Evaluando lo obtenido
# Funciones
def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['Not_bullying', 'Bullying']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

# Evaluando general
pd.DataFrame(hist.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

# Evaluando específico
input_text = 'Parecía carbón'
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")
!zip -r /content/file.zip /content/bullying_beto_modelo

y_pred = []
p=1
for i in X_test:
  #print(p)
  processed_data = prepare_data(i, tokenizer)
  result = make_prediction(sentiment_model, processed_data=processed_data)
  #print(f"Predicted Sentiment: {result}")
  if result=="Bullying":
    y_pred.append(1)
  else:
    y_pred.append(0)
  p+=1

# Para Excel
mod.append("BETO")
f1.append(round(f1_score(y_test,y_pred, average='weighted'),2))
acc.append(round(accuracy_score(y_test,y_pred),2))
auc.append(round(roc_auc_score(y_test,y_pred),2))
prec.append(round(precision_score(y_test,y_pred),2))
print(confusion_matrix(y_test, y_pred))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
spec.append(round(specificity,2))
sens.append(round(sensitivity,2))
vpps = tp / (tp+fp)
vpns = tn / (tn+fn)
vpp.append(round(vpps,2))
vpn.append(round(vpns,2))

### Comparativa

In [ ]:
datax = list(zip(mod,acc,prec,spec,sens,f1,auc,vpn))
columns = ["Modelo","Accuracy","Precision","Specificity","Sensivility","F1-score","AUC","FPV"]
datax = pd.DataFrame(datax,columns=columns)
datax.to_excel("aqui.xlsx",index=False)

### Calibración

In [ ]:
plt.clf()

# Calibración
print(nn.get_params())
from sklearn.model_selection import GridSearchCV

param_grid = {'learning_rate_init': [0.1, 0.3, 0.5, 0.7], 'solver': ['sgd', 'adam'], 'hidden_layer_sizes': [(150,100,50,10),(100,50,10)]}

scoring = ['f1_macro']
learning_rate=[0.1, 0.3, 0.5, 0.7]
solver = ['sgd', 'adam']
hls= [(150,100,50,10),(100,50,10)]

lr=[]
so=[]
hl=[]
f1=[]
acc=[]
auc=[]
prec=[]
spec=[]
sens=[]
vpp=[]
vpn=[]

for i in learning_rate:
  for j in solver:
    for k in hls:
      print("Probando con lr de", str(i), ", un solver de", str(j)," y un hls de", k)
      model=MLPClassifier(hidden_layer_sizes=k, max_iter=300, learning_rate_init=i, activation = 'relu', solver=j,random_state=17).fit(X_train,y_train)
      y_pred = model.predict(X_test)
      lr.append(i)
      f1.append(round(f1_score(y_test,y_pred, average='weighted'),2))
      acc.append(round(accuracy_score(y_test,y_pred),2))
      auc.append(round(roc_auc_score(y_test,y_pred),2))
      prec.append(round(precision_score(y_test,y_pred),2))
      tn_s, fp_s, fn_s, tp_s = confusion_matrix(y_test, y_pred).ravel()
      specificity_s = tn_s / (tn_s+fp_s)
      sensitivity_s = tp_s / (tp_s+fn_s)
      spec.append(round(specificity_s,2))
      sens.append(round(sensitivity_s,2))
      vpps_s = tp_s / (tp_s+fp_s)
      vpns_s = tn_s / (tn_s+fn_s)
      vpp.append(round(vpps_s,2))
      vpn.append(round(vpns_s,2))

datax = list(zip(lr,acc,prec,spec,sens,f1,auc,vpn))
columns = ["Learning rate","Accuracy","Precision","Specificity","Sensivility","F1-score","AUC","NPV"]
datax = pd.DataFrame(datax,columns=columns)
datax.to_excel("aqui2.xlsx",index=False)

#Inglés

### Estadística descriptiva

In [ ]:
# Modelando
f1=[]
acc=[]
prec=[]
spec=[]
sens=[]
vpp=[]
vpn=[]
auc=[]
mod=[]
dat=[]

# Longitud
text_len = []
for text in dff["tweet_text"]:
    tweet_len = len(text.split())
    text_len.append(tweet_len)

dff['tweet_text_len'] = text_len
plt.figure(figsize=(7,5))
ax = sns.countplot(x='tweet_text_len', data=dff, palette='magma')
plt.title('Conteo de textos originales por número de palabras (inglés)', fontsize=20)
plt.yticks([])
plt.xticks([])
#for i in ax.containers:
#    ax.bar_label(i,)
plt.ylabel('conteo')
plt.xlabel('')
plt.show()
print(dff['tweet_text_len'].describe())

from collections import Counter
results = set()
dff['tweet_text'].str.lower().str.split().apply(results.update)
results = Counter()
dff['tweet_text'].str.lower().str.split().apply(results.update)
print(len(results))
print(results.most_common())

### TF-IDF

In [ ]:
nlp = en_core_web_md.load()
documents = []
j = 0
X = dff[["Unnamed: 0",'tweet_text','tweet_text_translated','prediccion_jonatan']]
XX = X["tweet_text"]
y = yyy["cyberbullying_type2"]

if pp!=1:
  for sen in XX.index:
      # Removiendo caracteres especiales
      #print(X[sen])
      document = re.sub(r'\W', ' ', str(XX[sen]))
      
      # Removiendo caracteres solitarios despues del inicio
      document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
      
      # Removiendo caracteres solitarios al inicio
      document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

      # Removiendo multiples espacios
      document = re.sub(r'\s+', ' ', document, flags=re.I)

      # Removiendo el prefijo b
      document = re.sub(r'^b\s+', '', document) #solo cuando se agrega al inicio
      
      # Minúsculas
      document = document.lower()

      # Removiendo caracteres solitarios al inicio
      document = re.sub('rt', '', document) 

      # Removiendo caracteres solitarios al inicio
      document = re.sub('http', '', document)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('él', '', document)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('co', '', document)
      
      # Lematizacion
      document = nlp (document)
      lemmas = [no_stop.lemma_ for no_stop in document if not no_stop.is_stop]
      document = ' '.join(lemmas)
      documents.append(document)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('él', '', document)

      # Removiendo caracteres solitarios despues del inicio
      document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
      
      # Removiendo caracteres solitarios al inicio
      document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

      j = j+1
      print(j)

  # Longitud
  text_len = []
  for text in documents:
      tweet_len = len(text.split())
      text_len.append(tweet_len)

  dff["tweet_text_clean"]=documents
  dff['text_clean_len'] = text_len
  dff.to_excel("data_lista.xlsx",index=False)

plt.figure(figsize=(7,5))
ax = sns.countplot(x='text_translated_clean_len', data=dff, palette='magma')
plt.title('Conteo de textos limpiados por número de palabras (inglés)', fontsize=20)
plt.yticks([])
plt.xticks([])
#for i in ax.containers:
#    ax.bar_label(i,)
plt.ylabel('conteo')
plt.xlabel('')
plt.show()
print(dff['text_clean_len'].describe())

from collections import Counter
results = set()
dff['tweet_text_clean'].str.lower().str.split().apply(results.update)
results = Counter()
dff['tweet_text_clean'].str.lower().str.split().apply(results.update)
print(len(results))
print(results.most_common()[0:10])
yy = [count for tag, count in results.most_common(20)]
x = [tag for tag, count in results.most_common(20)]

plt.barh(x,yy, color='crimson')

# Vectorizar
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1000, min_df=5, max_df=0.7)
XX = vectorizer.fit_transform(dff['tweet_text_clean']).toarray()

# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
XX = tfidfconverter.fit_transform(XX).toarray()

# Separando (equivalente al de arriba por el random state)
X_train, X_test, y_train, y_test = train_test_split(XX, y, test_size=0.2,random_state=17)
print("Hay por procesar:", len(X_test))



###  + RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Perra loca"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de bullying:", rf.predict_proba(review1_features)[0,1]) #0 es por la primera obs, y 1 es por prob de posi

mod.append("TF-IDF + RF")
f1.append(round(f1_score(y_test,y_pred, average='weighted'),2))
acc.append(round(accuracy_score(y_test,y_pred),2))
auc.append(round(roc_auc_score(y_test,y_pred),2))
prec.append(round(precision_score(y_test,y_pred),2))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
spec.append(round(specificity,2))
sens.append(round(sensitivity,2))
vpps = tp / (tp+fp)
vpns = tn / (tn+fn)
vpp.append(round(vpps,2))
vpn.append(round(vpns,2))

### + SVM

In [ ]:
from sklearn.svm import SVC
sv = SVC(random_state=0)
sv.fit(X_train, y_train)
y_pred = sv.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Perra loca"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de bullying:", sv.predict(review1_features.toarray())) #0 es por la primera obs, y 1 es por prob de posi

mod.append("TF-IDF + SVM")
f1.append(round(f1_score(y_test,y_pred, average='weighted'),2))
acc.append(round(accuracy_score(y_test,y_pred),2))
auc.append(round(roc_auc_score(y_test,y_pred),2))
prec.append(round(precision_score(y_test,y_pred),2))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
spec.append(round(specificity,2))
sens.append(round(sensitivity,2))
vpps = tp / (tp+fp)
vpns = tn / (tn+fn)
vpp.append(round(vpps,2))
vpn.append(round(vpns,2))

### + Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Puta"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de bullying:", nb.predict_proba(review1_features.toarray())[0,1]) #0 es por la primera obs, y 1 es por prob de posi

mod.append("TF-IDF + Naive Bayes")
f1.append(round(f1_score(y_test,y_pred, average='weighted'),2))
acc.append(round(accuracy_score(y_test,y_pred),2))
auc.append(round(roc_auc_score(y_test,y_pred),2))
prec.append(round(precision_score(y_test,y_pred),2))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
spec.append(round(specificity,2))
sens.append(round(sensitivity,2))
vpps = tp / (tp+fp)
vpns = tn / (tn+fn)
vpp.append(round(vpps,2))
vpn.append(round(vpns,2))

### + Redes neuronales

In [ ]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes=(150,100,50,10), max_iter=300, activation = 'relu',solver='sgd',random_state=17)
nn.fit(X_train, y_train)
y_pred = nn.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Puta"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de bullying:", nn.predict_proba(review1_features.toarray())[0,1]) #0 es por la primera obs, y 1 es por prob de posi

mod.append("TF-IDF + Redes Neuronales")
f1.append(round(f1_score(y_test,y_pred, average='weighted'),2))
acc.append(round(accuracy_score(y_test,y_pred),2))
auc.append(round(roc_auc_score(y_test,y_pred),2))
prec.append(round(precision_score(y_test,y_pred),2))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
spec.append(round(specificity,2))
sens.append(round(sensitivity,2))
vpps = tp / (tp+fp)
vpns = tn / (tn+fn)
vpp.append(round(vpps,2))
vpn.append(round(vpns,2))

### BERT

In [ ]:
# Tutorial: https://www.youtube.com/watch?v=wp9BudYGZyA&t=1862s
### Preparación general
# Librerías para BERT
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
modelo='bert-base-cased'
variable_x="tweet_text"
nombre_modelo='bullying_bert_model'
z=16 # Batch size
epocas=3 # N° épocas

X = dff[["Unnamed: 0",'tweet_text','tweet_text_translated','prediccion_jonatan']]
# train-test-split para documents (textos sin íconos especiales ni doble espaciado)
X_train, X_test, y_train, y_test = train_test_split(X[variable_x], y, test_size=0.2,random_state=17)
print("Hay por procesar:", len(X_test))
print(X_test)

# Trayendo al modelo de Hugging Face
tokenizer = BertTokenizer.from_pretrained(modelo)

# Tokenizando
token = tokenizer.encode_plus(
    X_train.iloc[0],
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)
print(token.input_ids)
X_input_ids = np.zeros((len(X_train), 256))
X_attn_masks = np.zeros((len(X_train), 256))

# Funciones
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(X_train)):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

# Creando efectivamente la tokenización
X_input_ids, X_attn_masks = generate_training_data(X_train, X_input_ids, X_attn_masks, tokenizer)
labels = np.zeros((len(X_train), 2))
labels[np.arange(len(X_train)), y_train] = 1

# Pasando a tensorflow form
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
#dataset.take(2) # one sample data

def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

### Preparación modelo
# Seteando modelo
print("Batch size de:",z,"\nNúmero de épocas de:",epocas)
dataset = dataset.map(SentimentDatasetMapFunction)
print(len(dataset))
dataset = dataset.batch(z, drop_remainder=True)
print(len(dataset))
p = 0.8 # Porcentaje de datos para entrenar
train_size = int((len(X_train)//z)*p)
train_dataset = dataset.take(train_size)
print(len(train_dataset))
val_dataset = dataset.skip(train_size)
print(len(val_dataset))

from transformers import TFBertModel
model = TFBertModel.from_pretrained(modelo)

# Hay 2 inputs, los id y los attention mask
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

# Los embeddings
bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(2, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

# Arquitectura
sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

# Estimando modelo
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
accc = tf.keras.metrics.CategoricalAccuracy('accuracy')
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[accc])
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epocas
)

# Guardando
sentiment_model.save(nombre_modelo)

# Cargando
sentiment_model = tf.keras.models.load_model(nombre_modelo)
tokenizer = BertTokenizer.from_pretrained(modelo)

### Evaluando lo obtenido
# Funciones
def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['Not_bullying', 'Bullying']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

# Evaluando general
pd.DataFrame(hist.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

# Evaluando específico
input_text = 'You look like shit'
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")
!zip -r /content/file.zip /content/bullying_beto_modelo

y_pred = []
p=1
for i in X_test:
  #print(p)
  processed_data = prepare_data(i, tokenizer)
  result = make_prediction(sentiment_model, processed_data=processed_data)
  #print(f"Predicted Sentiment: {result}")
  if result=="Bullying":
    y_pred.append(1)
  else:
    y_pred.append(0)
  p+=1

# Para Excel
mod.append("BERT")
f1.append(round(f1_score(y_test,y_pred, average='weighted'),2))
acc.append(round(accuracy_score(y_test,y_pred),2))
auc.append(round(roc_auc_score(y_test,y_pred),2))
prec.append(round(precision_score(y_test,y_pred),2))
print(confusion_matrix(y_test, y_pred))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
spec.append(round(specificity,2))
sens.append(round(sensitivity,2))
vpps = tp / (tp+fp)
vpns = tn / (tn+fn)
vpp.append(round(vpps,2))
vpn.append(round(vpns,2))

In [ ]:
# Tutorial: https://www.youtube.com/watch?v=wp9BudYGZyA&t=1862s
### Preparación general
# Librerías para BERT
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
modelo='bert-base-cased'
variable_x="tweet_text_clean"
nombre_modelo='bullying_bert_model'
z=16 # Batch size
epocas=3 # N° épocas

X = dff[["Unnamed: 0",'tweet_text_clean','tweet_text_translated','prediccion_jonatan']]
# train-test-split para documents (textos sin íconos especiales ni doble espaciado)
X_train, X_test, y_train, y_test = train_test_split(X[variable_x], y, test_size=0.2,random_state=17)
print("Hay por procesar:", len(X_test))
print(X_test)

# Trayendo al modelo de Hugging Face
tokenizer = BertTokenizer.from_pretrained(modelo)

# Tokenizando
token = tokenizer.encode_plus(
    X_train.iloc[0],
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)
print(token.input_ids)
X_input_ids = np.zeros((len(X_train), 256))
X_attn_masks = np.zeros((len(X_train), 256))

# Funciones
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(X_train)):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

# Creando efectivamente la tokenización
X_input_ids, X_attn_masks = generate_training_data(X_train, X_input_ids, X_attn_masks, tokenizer)
labels = np.zeros((len(X_train), 2))
labels[np.arange(len(X_train)), y_train] = 1

# Pasando a tensorflow form
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
#dataset.take(2) # one sample data

def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

### Preparación modelo
# Seteando modelo
print("Batch size de:",z,"\nNúmero de épocas de:",epocas)
dataset = dataset.map(SentimentDatasetMapFunction)
print(len(dataset))
dataset = dataset.batch(z, drop_remainder=True)
print(len(dataset))
p = 0.8 # Porcentaje de datos para entrenar
train_size = int((len(X_train)//z)*p)
train_dataset = dataset.take(train_size)
print(len(train_dataset))
val_dataset = dataset.skip(train_size)
print(len(val_dataset))

from transformers import TFBertModel
model = TFBertModel.from_pretrained(modelo)

# Hay 2 inputs, los id y los attention mask
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

# Los embeddings
bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(2, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

# Arquitectura
sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

# Estimando modelo
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
accc = tf.keras.metrics.CategoricalAccuracy('accuracy')
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[accc])
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epocas
)

# Guardando
sentiment_model.save(nombre_modelo)

# Cargando
sentiment_model = tf.keras.models.load_model(nombre_modelo)
tokenizer = BertTokenizer.from_pretrained(modelo)

### Evaluando lo obtenido
# Funciones
def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['Not_bullying', 'Bullying']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

# Evaluando general
pd.DataFrame(hist.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

# Evaluando específico
input_text = 'You look like shit'
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")
!zip -r /content/file.zip /content/bullying_beto_modelo

y_pred = []
p=1
for i in X_test:
  #print(p)
  processed_data = prepare_data(i, tokenizer)
  result = make_prediction(sentiment_model, processed_data=processed_data)
  #print(f"Predicted Sentiment: {result}")
  if result=="Bullying":
    y_pred.append(1)
  else:
    y_pred.append(0)
  p+=1

# Para Excel
mod.append("BERT")
f1.append(round(f1_score(y_test,y_pred, average='weighted'),2))
acc.append(round(accuracy_score(y_test,y_pred),2))
auc.append(round(roc_auc_score(y_test,y_pred),2))
prec.append(round(precision_score(y_test,y_pred),2))
print(confusion_matrix(y_test, y_pred))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
spec.append(round(specificity,2))
sens.append(round(sensitivity,2))
vpps = tp / (tp+fp)
vpns = tn / (tn+fn)
vpp.append(round(vpps,2))
vpn.append(round(vpns,2))

### Comparativa

In [ ]:
datax = list(zip(mod,acc,prec,spec,sens,f1,auc,vpn))
columns = ["Modelo","Accuracy","Precision","Specificity","Sensivility","F1-score","AUC","FPV"]
datax = pd.DataFrame(datax,columns=columns)
datax.to_excel("aqui.xlsx",index=False)


### Calibración

In [ ]:
plt.clf()

# Calibración
print(nn.get_params())
from sklearn.model_selection import GridSearchCV

param_grid = {'learning_rate_init': [0.1, 0.3, 0.5, 0.7], 'solver': ['sgd', 'adam'], 'hidden_layer_sizes': [(150,100,50,10),(100,50,10)]}

scoring = ['f1_macro']
learning_rate=[0.1, 0.3, 0.5, 0.7]
solver = ['sgd', 'adam']
hls= [(150,100,50,10),(100,50,10)]

lr=[]
so=[]
hl=[]
f1=[]
acc=[]
auc=[]
prec=[]
spec=[]
sens=[]
vpp=[]
vpn=[]

for i in learning_rate:
  for j in solver:
    for k in hls:
      print("Probando con lr de", str(i), ", un solver de", str(j)," y un hls de", k)
      model=MLPClassifier(hidden_layer_sizes=k, max_iter=300, learning_rate_init=i, activation = 'relu', solver=j,random_state=17).fit(X_train,y_train)
      y_pred = model.predict(X_test)
      lr.append(i)
      f1.append(round(f1_score(y_test,y_pred, average='weighted'),2))
      acc.append(round(accuracy_score(y_test,y_pred),2))
      auc.append(round(roc_auc_score(y_test,y_pred),2))
      prec.append(round(precision_score(y_test,y_pred),2))
      tn_s, fp_s, fn_s, tp_s = confusion_matrix(y_test, y_pred).ravel()
      specificity_s = tn_s / (tn_s+fp_s)
      sensitivity_s = tp_s / (tp_s+fn_s)
      spec.append(round(specificity_s,2))
      sens.append(round(sensitivity_s,2))
      vpps_s = tp_s / (tp_s+fp_s)
      vpns_s = tn_s / (tn_s+fn_s)
      vpp.append(round(vpps_s,2))
      vpn.append(round(vpns_s,2))

datax = list(zip(lr,acc,prec,spec,sens,f1,auc,vpn))
columns = ["Learning rate","Accuracy","Precision","Specificity","Sensivility","F1-score","AUC","NPV"]
datax = pd.DataFrame(datax,columns=columns)
datax.to_excel("aqui2.xlsx",index=False)

# Multi clase

# Español

## Estadística descriptiva

In [ ]:
# Modelando
f1=[]
acc=[]
prec=[]
spec=[]
sens=[]
vpp=[]
vpn=[]
auc=[]
mod=[]
dat=[]

etiquetas=["No","Edad","Género","Etnicidad","Religión","Otro tipo"]

## TF-IDF

In [ ]:
nlp = es_core_news_md.load()
documents = []
j = 0
X = dff[["Unnamed: 0",'tweet_text','tweet_text_translated','prediccion_jonatan','tweet_text_translated_clean']]
XX = X["tweet_text_translated"]
y = yyy["cyberbullying_type3"]

if pp!=1:
  for sen in XX.index:
      # Removiendo caracteres especiales
      document = re.sub(r'\W', ' ', str(XX[sen]))
      
      # Removiendo caracteres solitarios despues del inicio
      document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
      
      # Removiendo caracteres solitarios al inicio
      document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

      # Removiendo el prefijo b
      document = re.sub(r'^b\s+', '', document) #solo cuando se agrega al inicio
      
      # Minúsculas
      document = document.lower()

      # Removiendo caracteres solitarios al inicio
      document = re.sub('rt', '', document)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('http', '', document)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('co', '', document)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('_', '', document) 

      # Removiendo multiples espacios
      document = re.sub(r'\s+', ' ', document, flags=re.I)
      
      # Lematizacion
      document = nlp (document)
      lemmas = [no_stop.lemma_ for no_stop in document if not no_stop.is_stop]
      document = ' '.join(lemmas)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('él', '', document)

      # Removiendo caracteres solitarios despues del inicio
      document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
      
      # Removiendo caracteres solitarios al inicio
      document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)
  
      documents.append(document)
      j = j+1
      print(j)

  # Longitud
  text_len = []
  for text in documents:
      tweet_len = len(text.split())
      text_len.append(tweet_len)

  dff["tweet_text_translated_clean"]=documents
  dff['text_translated_clean_len'] = text_len
  dff=dff.loc[~pd.isnull(dff["tweet_text_translated_clean"])]

plt.figure(figsize=(7,5))
ax = sns.countplot(x='text_translated_clean_len', data=dff, palette='magma')
plt.title('Conteo de textos limpiados por número de palabras (español)', fontsize=20)
plt.yticks([])
plt.xticks([])
#for i in ax.containers:
#    ax.bar_label(i,)
plt.ylabel('conteo')
plt.xlabel('')
plt.show()
print(dff['text_translated_clean_len'].describe())

from collections import Counter
results = set()
dff['tweet_text_translated_clean'].str.lower().str.split().apply(results.update)
results = Counter()
dff['tweet_text_translated_clean'].str.lower().str.split().apply(results.update)
print(len(results))
#print(results.most_common())

yy = [count for tag, count in results.most_common(20)]
x = [tag for tag, count in results.most_common(20)]

plt.barh(x,yy, color='crimson')

# Vectorizar
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1000, min_df=5, max_df=0.7)
XX = vectorizer.fit_transform(dff['tweet_text_translated_clean']).toarray()

# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
XX = tfidfconverter.fit_transform(XX).toarray()

# Separando (equivalente al de arriba por el random state)
X_train, X_test, y_train, y_test = train_test_split(XX, y, test_size=0.2,random_state=17)
print("Hay por procesar:", len(X_test))

## + RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
cooler_confusion_matrix(y_test,y_pred,"RF",etiquetas)
print(classification_report(y_test,y_pred, target_names=etiquetas))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Me gusta las finanzas"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de no bullying:", rf.predict_proba(review1_features)[0,0]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por edad:", rf.predict_proba(review1_features)[0,1]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por género:", rf.predict_proba(review1_features)[0,2]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por etnicidad:", rf.predict_proba(review1_features)[0,3]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por religión:", rf.predict_proba(review1_features)[0,4]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por otro tipo:", rf.predict_proba(review1_features)[0,5]) #0 es por la primera obs, y 0 es por prob de no bullying

## + SVM

In [ ]:
from sklearn.svm import SVC
sv = SVC(random_state=17)
sv.fit(X_train, y_train)
y_pred = sv.predict(X_test)
cooler_confusion_matrix(y_test,y_pred,"SVM",etiquetas)
print(classification_report(y_test,y_pred, target_names=etiquetas))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Perra loca"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de no bullying:", sv.predict(review1_features.toarray())) #0 es por la primera obs, y 0 es por prob de no bullying

## + Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
cooler_confusion_matrix(y_test,y_pred,"NB",etiquetas)
print(classification_report(y_test,y_pred, target_names=etiquetas))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Negra, a que hora vas por el pan?"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de no bullying:", nb.predict_proba(review1_features)[0,0]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por edad:", nb.predict_proba(review1_features)[0,1]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por género:", nb.predict_proba(review1_features)[0,2]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por etnicidad:", nb.predict_proba(review1_features)[0,3]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por religión:", nb.predict_proba(review1_features)[0,4]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por otro tipo:", nb.predict_proba(review1_features)[0,5]) #0 es por la primera obs, y 0 es por prob de no bullying

## + Redes Neuronales

In [ ]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes=(150,100,50,10), max_iter=300, activation = 'relu',solver='sgd',random_state=17)
nn.fit(X_train, y_train)
y_pred = nn.predict(X_test)
cooler_confusion_matrix(y_test,y_pred,"Redes Neuronales",etiquetas)
print(classification_report(y_test,y_pred, target_names=etiquetas))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Perra loca"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de no bullying:", nn.predict_proba(review1_features)[0,0]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por edad:", nn.predict_proba(review1_features)[0,1]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por género:", nn.predict_proba(review1_features)[0,2]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por etnicidad:", nn.predict_proba(review1_features)[0,3]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por religión:", nn.predict_proba(review1_features)[0,4]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por otro tipo:", nn.predict_proba(review1_features)[0,5]) #0 es por la primera obs, y 0 es por prob de no bullying

## Calibración

In [ ]:
plt.clf()

# Calibración
print(rf.get_params())
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': [3,5,7,10], 'n_estimators': [500,1000,1500], 'max_features': ['auto','sqrt']}

max_depth=[50,70,100,None]
n_estimators=[500,1000,1500]
max_features=['auto','sqrt']

for i in max_depth:
  for j in n_estimators:
    for k in max_features:
      print("Probando con max_depth de", str(i), ", un n_estimators de", str(j)," y un max_features", k)
      model=RandomForestClassifier(max_depth=i, n_estimators=j, max_features=k, random_state=0).fit(X_train,y_train)
      y_pred = model.predict(X_test)
      cooler_confusion_matrix(y_test,y_pred,"Modelo",etiquetas)
      print(classification_report(y_test,y_pred, target_names=etiquetas))


model=RandomForestClassifier(max_depth=None, n_estimators=1000, max_features='auto', random_state=0).fit(X_train,y_train)
y_pred = model.predict(X_test)
cooler_confusion_matrix(y_test,y_pred,"Modelo",etiquetas)
print(classification_report(y_test,y_pred, target_names=etiquetas))

## BETO

In [ ]:
# Tutorial: https://www.youtube.com/watch?v=wp9BudYGZyA&t=1862s
### Preparación general
# Librerías para BETO
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
modelo='dccuchile/bert-base-spanish-wwm-uncased'
variable_x="tweet_text_translated"
nombre_modelo='bullying_beto_modelo'
z=16 # Batch size
epocas=3 # N° épocas

X = dff[["Unnamed: 0",'tweet_text','tweet_text_translated','prediccion_jonatan']]
# train-test-split para documents (textos sin íconos especiales ni doble espaciado)
X_train, X_test, y_train, y_test = train_test_split(X[variable_x], y, test_size=0.2,random_state=17)
print("Hay por procesar:", len(X_test))
print(X_test)

# Trayendo al modelo de Hugging Face
tokenizer = BertTokenizer.from_pretrained(modelo)

# Tokenizando
token = tokenizer.encode_plus(
    X_train.iloc[0],
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)
print(token.input_ids)
X_input_ids = np.zeros((len(X_train), 256))
X_attn_masks = np.zeros((len(X_train), 256))

# Funciones
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(X_train)):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

# Creando efectivamente la tokenización
X_input_ids, X_attn_masks = generate_training_data(X_train, X_input_ids, X_attn_masks, tokenizer)
labels = np.zeros((len(X_train), 6))
labels[np.arange(len(X_train)), y_train] = 1

# Pasando a tensorflow form
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
#dataset.take(2) # one sample data

def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

### Preparación modelo
# Seteando modelo
print("Batch size de:",z,"\nNúmero de épocas de:",epocas)
dataset = dataset.map(SentimentDatasetMapFunction)
print(len(dataset))
dataset = dataset.batch(z, drop_remainder=True)
print(len(dataset))
p = 0.8 # Porcentaje de datos para entrenar
train_size = int((len(X_train)//z)*p)
train_dataset = dataset.take(train_size)
print(len(train_dataset))
val_dataset = dataset.skip(train_size)
print(len(val_dataset))

from transformers import TFBertModel
model = TFBertModel.from_pretrained(modelo)

# Hay 2 inputs, los id y los attention mask
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

# Los embeddings
bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(6, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

# Arquitectura
sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

# Estimando modelo
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
accc = tf.keras.metrics.CategoricalAccuracy('accuracy')
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[accc])
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epocas
)

# Guardando
sentiment_model.save(nombre_modelo)

# Cargando
sentiment_model = tf.keras.models.load_model(nombre_modelo)
tokenizer = BertTokenizer.from_pretrained(modelo)

### Evaluando lo obtenido
# Funciones
def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=etiquetas):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

# Evaluando general
pd.DataFrame(hist.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

# Evaluando específico
input_text = 'Parecía carbón'
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")
#!zip -r /content/file.zip /content/bullying_beto_modelo

y_pred = []
p=1
for i in X_test:
  #print(p)
  processed_data = prepare_data(i, tokenizer)
  result = make_prediction(sentiment_model, processed_data=processed_data)
  #print(f"Predicted Sentiment: {result}")
  if result=="No":
    y_pred.append(0)
  elif result=="Edad":
    y_pred.append(1)
  elif result=="Género":
    y_pred.append(2)
  elif result=="Etnicidad":
    y_pred.append(3)
  elif result=="Religión":
    y_pred.append(4)
  elif result=="Otro tipo":
    y_pred.append(5)
  p+=1

# Para Excel
cooler_confusion_matrix(y_test,y_pred,"BETO normal",etiquetas)
print(classification_report(y_test,y_pred, target_names=etiquetas))

In [ ]:
# Tutorial: https://www.youtube.com/watch?v=wp9BudYGZyA&t=1862s
### Preparación general
# Librerías para BETO
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
modelo='dccuchile/bert-base-spanish-wwm-uncased'
variable_x="tweet_text_translated_clean"
nombre_modelo='bullying_beto_modelo'
z=16 # Batch size
epocas=3 # N° épocas

X = dff[["Unnamed: 0",'tweet_text','tweet_text_translated','tweet_text_translated_clean','prediccion_jonatan']]
# train-test-split para documents (textos sin íconos especiales ni doble espaciado)
X_train, X_test, y_train, y_test = train_test_split(X[variable_x], y, test_size=0.2,random_state=17)
print("Hay por procesar:", len(X_test))
print(X_test)

# Trayendo al modelo de Hugging Face
tokenizer = BertTokenizer.from_pretrained(modelo)

# Tokenizando
token = tokenizer.encode_plus(
    X_train.iloc[0],
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)
print(token.input_ids)
X_input_ids = np.zeros((len(X_train), 256))
X_attn_masks = np.zeros((len(X_train), 256))

# Funciones
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(X_train)):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

# Creando efectivamente la tokenización
X_input_ids, X_attn_masks = generate_training_data(X_train, X_input_ids, X_attn_masks, tokenizer)
labels = np.zeros((len(X_train), 6))
labels[np.arange(len(X_train)), y_train] = 1

# Pasando a tensorflow form
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
#dataset.take(2) # one sample data

def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

### Preparación modelo
# Seteando modelo
print("Batch size de:",z,"\nNúmero de épocas de:",epocas)
dataset = dataset.map(SentimentDatasetMapFunction)
print(len(dataset))
dataset = dataset.batch(z, drop_remainder=True)
print(len(dataset))
p = 0.8 # Porcentaje de datos para entrenar
train_size = int((len(X_train)//z)*p)
train_dataset = dataset.take(train_size)
print(len(train_dataset))
val_dataset = dataset.skip(train_size)
print(len(val_dataset))

from transformers import TFBertModel
model = TFBertModel.from_pretrained(modelo)

# Hay 2 inputs, los id y los attention mask
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

# Los embeddings
bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(6, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

# Arquitectura
sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

# Estimando modelo
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
accc = tf.keras.metrics.CategoricalAccuracy('accuracy')
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[accc])
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epocas
)

# Guardando
sentiment_model.save(nombre_modelo)

# Cargando
sentiment_model = tf.keras.models.load_model(nombre_modelo)
tokenizer = BertTokenizer.from_pretrained(modelo)

### Evaluando lo obtenido
# Funciones
def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=etiquetas):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

# Evaluando general
pd.DataFrame(hist.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

# Evaluando específico
input_text = 'Parecía carbón'
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")
#!zip -r /content/file.zip /content/bullying_beto_modelo

y_pred = []
p=1
for i in X_test:
  #print(p)
  processed_data = prepare_data(i, tokenizer)
  result = make_prediction(sentiment_model, processed_data=processed_data)
  #print(f"Predicted Sentiment: {result}")
  if result=="No":
    y_pred.append(0)
  elif result=="Edad":
    y_pred.append(1)
  elif result=="Género":
    y_pred.append(2)
  elif result=="Etnicidad":
    y_pred.append(3)
  elif result=="Religión":
    y_pred.append(4)
  elif result=="Otro tipo":
    y_pred.append(5)
  p+=1

# Para Excel
cooler_confusion_matrix(y_test,y_pred,"BETO clean",etiquetas)
print(classification_report(y_test,y_pred, target_names=etiquetas))

# Inglés

## Estadística descriptiva

In [ ]:
# Modelando
f1=[]
acc=[]
prec=[]
spec=[]
sens=[]
vpp=[]
vpn=[]
auc=[]
mod=[]
dat=[]

etiquetas=["No","Edad","Género","Etnicidad","Religión","Otro tipo"]

## TF - IDF

In [ ]:
nlp = en_core_web_md.load()
documents = []
j = 0
X = dff[["Unnamed: 0",'tweet_text','tweet_text_translated','prediccion_jonatan']]
XX = X["tweet_text"]
y = yyy["cyberbullying_type3"]

if pp!=1:
  for sen in XX.index:
      # Removiendo caracteres especiales
      document = re.sub(r'\W', ' ', str(XX[sen]))
      
      # Removiendo caracteres solitarios despues del inicio
      document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
      
      # Removiendo caracteres solitarios al inicio
      document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

      # Removiendo el prefijo b
      document = re.sub(r'^b\s+', '', document) #solo cuando se agrega al inicio
      
      # Minúsculas
      document = document.lower()

      # Removiendo caracteres solitarios al inicio
      document = re.sub('rt', '', document)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('http', '', document)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('co', '', document)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('_', '', document) 

      # Removiendo multiples espacios
      document = re.sub(r'\s+', ' ', document, flags=re.I)
      
      # Lematizacion
      document = nlp (document)
      lemmas = [no_stop.lemma_ for no_stop in document if not no_stop.is_stop]
      document = ' '.join(lemmas)

      # Removiendo caracteres solitarios al inicio
      document = re.sub('él', '', document)

      # Removiendo caracteres solitarios despues del inicio
      document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
      
      # Removiendo caracteres solitarios al inicio
      document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)
  
      documents.append(document)
      j = j+1
      print(j)

  # Longitud
  text_len = []
  for text in documents:
      tweet_len = len(text.split())
      text_len.append(tweet_len)

  dff["tweet_text_translated_clean"]=documents
  dff['text_translated_clean_len'] = text_len
  dff=dff.loc[~pd.isnull(dff["tweet_text_translated_clean"])]

# Vectorizar
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1000, min_df=5, max_df=0.7)
XX = vectorizer.fit_transform(dff['tweet_text_clean']).toarray()

# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
XX = tfidfconverter.fit_transform(XX).toarray()

# Separando (equivalente al de arriba por el random state)
X_train, X_test, y_train, y_test = train_test_split(XX, y, test_size=0.2,random_state=17)
print("Hay por procesar:", len(X_test))

## + RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
cooler_confusion_matrix(y_test,y_pred,"RF",etiquetas)
print(classification_report(y_test,y_pred, target_names=etiquetas))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Me gusta las finanzas"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de no bullying:", rf.predict_proba(review1_features)[0,0]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por edad:", rf.predict_proba(review1_features)[0,1]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por género:", rf.predict_proba(review1_features)[0,2]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por etnicidad:", rf.predict_proba(review1_features)[0,3]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por religión:", rf.predict_proba(review1_features)[0,4]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por otro tipo:", rf.predict_proba(review1_features)[0,5]) #0 es por la primera obs, y 0 es por prob de no bullying

## + SVM

In [ ]:
from sklearn.svm import SVC
sv = SVC(random_state=17)
sv.fit(X_train, y_train)
y_pred = sv.predict(X_test)
cooler_confusion_matrix(y_test,y_pred,"SVM",etiquetas)
print(classification_report(y_test,y_pred, target_names=etiquetas))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Fuck you bitch"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de no bullying:", sv.predict(review1_features.toarray())) #0 es por la primera obs, y 0 es por prob de no bullying

## + Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
cooler_confusion_matrix(y_test,y_pred,"NB",etiquetas)
print(classification_report(y_test,y_pred, target_names=etiquetas))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Negra, a que hora vas por el pan?"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de no bullying:", nb.predict_proba(review1_features)[0,0]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por edad:", nb.predict_proba(review1_features)[0,1]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por género:", nb.predict_proba(review1_features)[0,2]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por etnicidad:", nb.predict_proba(review1_features)[0,3]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por religión:", nb.predict_proba(review1_features)[0,4]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por otro tipo:", nb.predict_proba(review1_features)[0,5]) #0 es por la primera obs, y 0 es por prob de no bullying

## + Redes Neuronales

In [ ]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes=(150,100,50,10), max_iter=300, activation = 'relu',solver='sgd',random_state=17)
nn.fit(X_train, y_train)
y_pred = nn.predict(X_test)
cooler_confusion_matrix(y_test,y_pred,"Redes Neuronales",etiquetas)
print(classification_report(y_test,y_pred, target_names=etiquetas))

# Escribe tu propia reseña y prueba tu modelo
review1 = "Perra loca"
review1_features = get_features(review1)
print("Review:", review1)
print("Probabilidad de no bullying:", nn.predict_proba(review1_features)[0,0]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por edad:", nn.predict_proba(review1_features)[0,1]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por género:", nn.predict_proba(review1_features)[0,2]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por etnicidad:", nn.predict_proba(review1_features)[0,3]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por religión:", nn.predict_proba(review1_features)[0,4]) #0 es por la primera obs, y 0 es por prob de no bullying
print("Probabilidad de bullying por otro tipo:", nn.predict_proba(review1_features)[0,5]) #0 es por la primera obs, y 0 es por prob de no bullying

## Calibración

In [ ]:
plt.clf()

# Calibración
print(rf.get_params())
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': [3,5,7,10], 'n_estimators': [500,1000,1500], 'max_features': ['auto','sqrt']}

max_depth=[50,70,100,None]
n_estimators=[500,1000,1500]
max_features=['auto','sqrt']

for i in max_depth:
  for j in n_estimators:
    for k in max_features:
      print("Probando con max_depth de", str(i), ", un n_estimators de", str(j)," y un max_features", k)
      model=RandomForestClassifier(max_depth=i, n_estimators=j, max_features=k, random_state=0).fit(X_train,y_train)
      y_pred = model.predict(X_test)
      cooler_confusion_matrix(y_test,y_pred,"Modelo",etiquetas)
      print(classification_report(y_test,y_pred, target_names=etiquetas))


model=RandomForestClassifier(max_depth=None, n_estimators=1000, max_features='auto', random_state=0).fit(X_train,y_train)
y_pred = model.predict(X_test)
cooler_confusion_matrix(y_test,y_pred,"Modelo",etiquetas)
print(classification_report(y_test,y_pred, target_names=etiquetas))

## BERT

In [ ]:
# Tutorial: https://www.youtube.com/watch?v=wp9BudYGZyA&t=1862s
### Preparación general
# Librerías para BETO
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
modelo='bert-base-cased'
variable_x="tweet_text"
nombre_modelo='bullying_bert_model'
z=16 # Batch size
epocas=3 # N° épocas

X = dff[["Unnamed: 0",'tweet_text','tweet_text_translated','prediccion_jonatan','tweet_text_clean']]
# train-test-split para documents (textos sin íconos especiales ni doble espaciado)
X_train, X_test, y_train, y_test = train_test_split(X[variable_x], y, test_size=0.2,random_state=17)
print("Hay por procesar:", len(X_test))
print(X_test)

# Trayendo al modelo de Hugging Face
tokenizer = BertTokenizer.from_pretrained(modelo)

# Tokenizando
token = tokenizer.encode_plus(
    X_train.iloc[0],
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)
print(token.input_ids)
X_input_ids = np.zeros((len(X_train), 256))
X_attn_masks = np.zeros((len(X_train), 256))

# Funciones
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(X_train)):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

# Creando efectivamente la tokenización
X_input_ids, X_attn_masks = generate_training_data(X_train, X_input_ids, X_attn_masks, tokenizer)
labels = np.zeros((len(X_train), 6))
labels[np.arange(len(X_train)), y_train] = 1

# Pasando a tensorflow form
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
#dataset.take(2) # one sample data

def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

### Preparación modelo
# Seteando modelo
print("Batch size de:",z,"\nNúmero de épocas de:",epocas)
dataset = dataset.map(SentimentDatasetMapFunction)
print(len(dataset))
dataset = dataset.batch(z, drop_remainder=True)
print(len(dataset))
p = 0.8 # Porcentaje de datos para entrenar
train_size = int((len(X_train)//z)*p)
train_dataset = dataset.take(train_size)
print(len(train_dataset))
val_dataset = dataset.skip(train_size)
print(len(val_dataset))

from transformers import TFBertModel
model = TFBertModel.from_pretrained(modelo)

# Hay 2 inputs, los id y los attention mask
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

# Los embeddings
bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(6, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

# Arquitectura
sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

# Estimando modelo
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
accc = tf.keras.metrics.CategoricalAccuracy('accuracy')
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[accc])
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epocas
)

# Guardando
sentiment_model.save(nombre_modelo)

# Cargando
sentiment_model = tf.keras.models.load_model(nombre_modelo)
tokenizer = BertTokenizer.from_pretrained(modelo)

### Evaluando lo obtenido
# Funciones
def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=etiquetas):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

# Evaluando general
pd.DataFrame(hist.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

# Evaluando específico
input_text = 'What a nice day'
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")
#!zip -r /content/file.zip /content/bullying_beto_modelo

y_pred = []
p=1
for i in X_test:
  #print(p)
  processed_data = prepare_data(i, tokenizer)
  result = make_prediction(sentiment_model, processed_data=processed_data)
  #print(f"Predicted Sentiment: {result}")
  if result=="No":
    y_pred.append(0)
  elif result=="Edad":
    y_pred.append(1)
  elif result=="Género":
    y_pred.append(2)
  elif result=="Etnicidad":
    y_pred.append(3)
  elif result=="Religión":
    y_pred.append(4)
  elif result=="Otro tipo":
    y_pred.append(5)
  p+=1

# Para Excel
cooler_confusion_matrix(y_test,y_pred,"BERT normal",etiquetas)
print(classification_report(y_test,y_pred, target_names=etiquetas))

In [ ]:
# Tutorial: https://www.youtube.com/watch?v=wp9BudYGZyA&t=1862s
### Preparación general
# Librerías para BETO
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
modelo='bert-base-cased'
variable_x="tweet_text_clean"
nombre_modelo='bullying_bert_model'
z=16 # Batch size
epocas=3 # N° épocas

X = dff[["Unnamed: 0",'tweet_text','tweet_text_translated','prediccion_jonatan','tweet_text_clean']]
# train-test-split para documents (textos sin íconos especiales ni doble espaciado)
X_train, X_test, y_train, y_test = train_test_split(X[variable_x], y, test_size=0.2,random_state=17)
print("Hay por procesar:", len(X_test))
print(X_test)

# Trayendo al modelo de Hugging Face
tokenizer = BertTokenizer.from_pretrained(modelo)

# Tokenizando
token = tokenizer.encode_plus(
    X_train.iloc[0],
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)
print(token.input_ids)
X_input_ids = np.zeros((len(X_train), 256))
X_attn_masks = np.zeros((len(X_train), 256))

# Funciones
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(X_train)):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

# Creando efectivamente la tokenización
X_input_ids, X_attn_masks = generate_training_data(X_train, X_input_ids, X_attn_masks, tokenizer)
labels = np.zeros((len(X_train), 6))
labels[np.arange(len(X_train)), y_train] = 1

# Pasando a tensorflow form
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
#dataset.take(2) # one sample data

def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

### Preparación modelo
# Seteando modelo
print("Batch size de:",z,"\nNúmero de épocas de:",epocas)
dataset = dataset.map(SentimentDatasetMapFunction)
print(len(dataset))
dataset = dataset.batch(z, drop_remainder=True)
print(len(dataset))
p = 0.8 # Porcentaje de datos para entrenar
train_size = int((len(X_train)//z)*p)
train_dataset = dataset.take(train_size)
print(len(train_dataset))
val_dataset = dataset.skip(train_size)
print(len(val_dataset))

from transformers import TFBertModel
model = TFBertModel.from_pretrained(modelo)

# Hay 2 inputs, los id y los attention mask
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

# Los embeddings
bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(6, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

# Arquitectura
sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

# Estimando modelo
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
accc = tf.keras.metrics.CategoricalAccuracy('accuracy')
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[accc])
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epocas
)

# Guardando
sentiment_model.save(nombre_modelo)

# Cargando
sentiment_model = tf.keras.models.load_model(nombre_modelo)
tokenizer = BertTokenizer.from_pretrained(modelo)

### Evaluando lo obtenido
# Funciones
def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=etiquetas):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

# Evaluando general
pd.DataFrame(hist.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

# Evaluando específico
input_text = 'That cristian girl is a bitch'
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")
#!zip -r /content/file.zip /content/bullying_beto_modelo

y_pred = []
p=1
for i in X_test:
  #print(p)
  processed_data = prepare_data(i, tokenizer)
  result = make_prediction(sentiment_model, processed_data=processed_data)
  #print(f"Predicted Sentiment: {result}")
  if result=="No":
    y_pred.append(0)
  elif result=="Edad":
    y_pred.append(1)
  elif result=="Género":
    y_pred.append(2)
  elif result=="Etnicidad":
    y_pred.append(3)
  elif result=="Religión":
    y_pred.append(4)
  elif result=="Otro tipo":
    y_pred.append(5)
  p+=1

# Para Excel
cooler_confusion_matrix(y_test,y_pred,"BERT clean",etiquetas)
print(classification_report(y_test,y_pred, target_names=etiquetas))